# Reorganising multiplexed zarr

To better match OME NGFF standards

1. First reorganise into main multiplexed.zarr
2. Read metadata and assay layout
3. Collate all information and save out metadata

In [92]:
import zarr
import shutil
import os
import json
from tqdm.auto import tqdm
import numpy as np
from macrohet import dataio
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import ast

def safe(x):
    "for the sanitisation of values into zattrs"
    return None if pd.isna(x) else x


def load_assay_layout(xml_file_name):
    """
    Loads and parses an assay layout XML file into a pandas DataFrame with a 'position' index.

    Args:
        xml_file_name (str): The full path to the assay layout XML file.

    Returns:
        pandas.DataFrame: A DataFrame containing the assay layout information,
                          with a 'position' column as index and each layer as a column.
                          Returns an empty DataFrame if the file is not found or parsing fails.
    """
    df_assay_layout = pd.DataFrame()

    if not os.path.exists(xml_file_name):
        print(f"Error: The file '{xml_file_name}' was not found.")
    else:
        try:
            tree = ET.parse(xml_file_name)
            root = tree.getroot()
            namespaces = {'ns': 'http://www.perkinelmer.com/PEHH/HarmonyV5'}
            plate_data = {}

            for layer in root.findall('ns:Layer', namespaces):
                layer_name_element = layer.find('ns:Name', namespaces)
                if layer_name_element is None or layer_name_element.text is None:
                    print("Warning: Found a layer without a name. Skipping this layer.")
                    continue
                layer_name = layer_name_element.text.strip()

                for well in layer.findall('ns:Well', namespaces):
                    row_element = well.find('ns:Row', namespaces)
                    col_element = well.find('ns:Col', namespaces)
                    value_element = well.find('ns:Value', namespaces)

                    if row_element is None or row_element.text is None or \
                       col_element is None or col_element.text is None:
                        print(f"Warning: Found a well in layer '{layer_name}' without row or col. Skipping this well.")
                        continue

                    try:
                        row = int(row_element.text.strip())
                        col = int(col_element.text.strip())
                    except ValueError:
                        print(f"Warning: Could not parse row/col as integers for a well in layer '{layer_name}'. Skipping this well.")
                        continue

                    value = value_element.text.strip() if value_element is not None and value_element.text is not None else pd.NA
                    well_key = (row, col)
                    if well_key not in plate_data:
                        plate_data[well_key] = {}
                    plate_data[well_key][layer_name] = value

            if plate_data:
                df_assay_layout = pd.DataFrame.from_dict(plate_data, orient='index')
                df_assay_layout.index.names = ['Row', 'Col']
                df_assay_layout = df_assay_layout.sort_index()
                df_assay_layout = df_assay_layout.dropna(how='all')

                # Create a new 'position' column
                df_assay_layout['position'] = df_assay_layout.index.map(lambda rc: f"({rc[0]}, {rc[1]})")
                # Set 'position' as the new index
                df_assay_layout = df_assay_layout.set_index('position')
                df_assay_layout = df_assay_layout.drop(columns=['Row', 'Col']) # Drop the old index columns

            else:
                print("No data was extracted from the XML file to populate the DataFrame.")

        except ET.ParseError as e:
            print(f"Error parsing XML file '{xml_file_name}': {e}")
        except FileNotFoundError:
            print(f"Error: The file '{xml_file_name}' was not found.")
        except Exception as e:
            print(f"An unexpected error occurred during XML processing: {e}")

    return df_assay_layout

### Reorganise the multiplexed data first

In [12]:
# Paths
source_root = "/mnt/DATA3/BPP0050/multiplexed"             # folder with *_plexed.zarr
destination_root = "/mnt/DATA3/BPP0050/multiplexed.zarr"   # new NGFF-compliant dataset

# Create destination root
os.makedirs(destination_root, exist_ok=True)

# Loop over each plexed zarr directory
for name in tqdm(os.listdir(source_root)):
    if name.endswith("_plexed.zarr"):
        fov_name = name.replace("_plexed.zarr", "")  # e.g., "2_4"
        src = os.path.join(source_root, name)
        dst = os.path.join(destination_root, fov_name)
        dst_img = os.path.join(dst, "images")

        print(f"Processing: {name} → {fov_name}")

        os.makedirs(dst_img, exist_ok=True)

        # Move chunk files and Zarr metadata to 'images'
        for fname in os.listdir(src):
            src_f = os.path.join(src, fname)
            dst_f = os.path.join(dst_img, fname) # if fname.startswith("0.") or fname in [".zarray", ".zattrs"] else os.path.join(dst, fname)

            if os.path.exists(dst_f):
                print(f"Skipping existing {dst_f}")
                continue

            shutil.move(src_f, dst_f)

        # Write new .zattrs in FOV root
        zattrs_fov = {
            "multiscales": [{
                "version": "0.4",
                "datasets": [{"path": "images"}],
                "axes": ["round", "time", "channel", "z", "y", "x"],
                "name": fov_name
            }],
            "axes": [
                {"name": "round", "type": "other"},
                {"name": "time", "type": "time"},
                {"name": "channel", "type": "channel"},
                {"name": "z", "type": "space"},
                {"name": "y", "type": "space"},
                {"name": "x", "type": "space"}
            ]
        }
        with open(os.path.join(dst, ".zattrs"), "w") as f:
            json.dump(zattrs_fov, f, indent=2)

  0%|          | 0/21 [00:00<?, ?it/s]

Processing: 2_8_plexed.zarr → 2_8
Processing: 3_7_plexed.zarr → 3_7
Processing: 3_8_plexed.zarr → 3_8
Processing: 3_4_plexed.zarr → 3_4
Processing: 4_4_plexed.zarr → 4_4
Processing: 4_8_plexed.zarr → 4_8
Processing: 4_7_plexed.zarr → 4_7
Processing: 4_6_plexed.zarr → 4_6
Processing: 3_9_plexed.zarr → 3_9
Processing: 2_4_plexed.zarr → 2_4
Processing: 2_6_plexed.zarr → 2_6
Processing: 3_5_plexed.zarr → 3_5
Processing: 4_5_plexed.zarr → 4_5
Processing: 3_6_plexed.zarr → 3_6
Processing: 2_7_plexed.zarr → 2_7
Processing: 5_5_plexed.zarr → 5_5
Processing: 4_9_plexed.zarr → 4_9
Processing: 3_2_plexed.zarr → 3_2
Processing: 3_3_plexed.zarr → 3_3
Processing: 5_4_plexed.zarr → 5_4
Processing: 2_5_plexed.zarr → 2_5


### Then load metadata

In [19]:
base_dir =  '/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy6__2025-04-19T16_14_26-Measurement 1'

metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  


Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


In [20]:
metadata.keys()

Index(['id', 'State', 'URL', 'Row', 'Col', 'FieldID', 'PlaneID', 'TimepointID',
       'ChannelID', 'FlimID', 'ChannelName', 'ImageType', 'AcquisitionType',
       'IlluminationType', 'ChannelType', 'ImageResolutionX',
       'ImageResolutionY', 'ImageSizeX', 'ImageSizeY', 'BinningX', 'BinningY',
       'MaxIntensity', 'CameraType', 'PositionX', 'PositionY', 'PositionZ',
       'AbsPositionZ', 'MeasurementTimeOffset', 'AbsTime',
       'MainExcitationWavelength', 'MainEmissionWavelength',
       'ObjectiveMagnification', 'ObjectiveNA', 'ExposureTime',
       'OrientationMatrix'],
      dtype='object')

In [86]:

metadata_fn = glob.glob('/mnt/DATA3/BPP0050/BPP0050-1-Live-cell-to4i_Fixed_Cy6__2025-04-19T16_14_26-Measurement 1/acquisition/Assaylayout/*xml*')[0]
assay_layout = load_assay_layout(metadata_fn)  


An unexpected error occurred during XML processing: "['Row', 'Col'] not found in axis"


In [31]:
assay_layout

,Infection status,Antibiotic treatment
position,,
"(2, 4)",None,None
"(2, 5)",None,PZA EC99
"(2, 6)",None,RIF EC99
"(2, 7)",None,INH EC99
"(2, 8)",None,BDQ EC99
"(3, 2)",None,None
"(3, 3)",None,None
"(3, 4)",MtbWT,None
"(3, 5)",MtbWT,PZA EC99


### Parse the metadata info

In [55]:
# Extract Z spacing from PositionZ
z_um = metadata['PositionZ'].astype(float) * 1e6
z_unique = np.sort(z_um.dropna().unique())
z_spacing_um = float(np.mean(np.diff(z_unique)))
y_spacing_um = float(metadata['ImageResolutionY'].astype(float).iloc[0] * 1e6)
x_spacing_um = float(metadata['ImageResolutionX'].astype(float).iloc[0] * 1e6)


In [90]:
# Group metadata by (Row, Col, ChannelID) to extract channel-specific info
channel_grouped = (
    metadata.groupby(["Row", "Col", "ChannelID"])
    .agg({
        "ChannelName": "first",
        "ExposureTime": "first",
        "ImageResolutionX": "first",
        "ImageResolutionY": "first",
        "ImageType": "first",
        "ChannelType": "first",
        "IlluminationType": "first",
        "AcquisitionType": "first",
        "CameraType": "first",
        "BinningX": "first",
        "BinningY": "first",
        "MainExcitationWavelength": "first",
        "MainEmissionWavelength": "first",
        "ObjectiveMagnification": "first",
        "ObjectiveNA": "first"
    })
    .reset_index()
)

# Extract Z spacing from PositionZ
z_um = metadata['PositionZ'].astype(float) * 1e6
z_unique = np.sort(z_um.dropna().unique())
z_spacing_um = float(np.mean(np.diff(z_unique)))
y_spacing_um = float(metadata['ImageResolutionY'].astype(float).iloc[0] * 1e6)
x_spacing_um = float(metadata['ImageResolutionX'].astype(float).iloc[0] * 1e6)

# Iterate through FOVs
fovs = channel_grouped.groupby(["Row", "Col"])

metadata_dir = "/mnt/DATA3/BPP0050/multiplexed.zarr"

for (row, col), group in fovs:
    fov_path = os.path.join(metadata_dir, f"{row}_{col}")
    zattrs_path = os.path.join(fov_path, ".zattrs")
    os.makedirs(fov_path, exist_ok=True)

    # Channel metadata
    channels = []
    for _, ch_row in group.iterrows():
        channels.append({
            "id": int(ch_row["ChannelID"]),
            "name": ch_row["ChannelName"],
            "exposure_time": ch_row["ExposureTime"],
            "excitation_wavelength": ch_row["MainExcitationWavelength"],
            "emission_wavelength": ch_row["MainEmissionWavelength"],
            "channel_type": ch_row["ChannelType"]
        })

    # Experimental conditions
    try:
        condition = assay_layout.loc[str((int(row), int(col)))]
        infection_status = condition.get("Infection status", None)
        antibiotic_treatment = condition.get("Antibiotic treatment", None)
    except KeyError:
        infection_status = None
        antibiotic_treatment = None

    zattrs = {
        "multiscales": [
            {
                "version": "0.4",
                "datasets": [
                    {
                        "path": "images",
                        "coordinateTransformations": [
                            {
                                "type": "scale",
                                "scale": [1, 1, 1, z_spacing_um, y_spacing_um, x_spacing_um]
                            }
                        ]
                    }
                ],
                "axes": [
                    {"name": "round", "type": "other"},
                    {"name": "time", "type": "time"},
                    {"name": "channel", "type": "channel"},
                    {"name": "z", "type": "space", "unit": "micrometer"},
                    {"name": "y", "type": "space", "unit": "micrometer"},
                    {"name": "x", "type": "space", "unit": "micrometer"}
                ]
            }
        ],
        "acquisition_metadata": {
            "infection_status": safe(infection_status),
            "antibiotic_treatment": safe(antibiotic_treatment),
            "objective_magnification": group["ObjectiveMagnification"].iloc[0],
            "objective_na": group["ObjectiveNA"].iloc[0],
            "camera": group["CameraType"].iloc[0],
            "binning": {
                "x": group["BinningX"].iloc[0],
                "y": group["BinningY"].iloc[0]
            },
            "image_type": group["ImageType"].iloc[0],
            "illumination_type": group["IlluminationType"].iloc[0],
            "acquisition_type": group["AcquisitionType"].iloc[0],
            "channels": channels
        }
    }

    # Write to .zattrs
    with open(zattrs_path, "w") as f:
        json.dump(zattrs, f, indent=2)


## Now do the same for the live-cell

wait until its finished compiling the arrays